### Импорты

In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import mutual_info_classif
from catboost import CatBoostClassifier, Pool, cv
from sklearn.preprocessing import StandardScaler


import matplotlib.pyplot as plt
import seaborn as sns


### Загрузка данных

In [3]:
data = pd.read_csv('../data/train_with_all_features_made_without_cat_encoding.csv')
data.head()

,home_ownership,income,years_in_current_job,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,number_of_credit_problem,bankruptcies,purpose,...,number_of_bankruptcies_per_history,current_credit_balance_per_income,monthly_income_per_monthly_debt,current_loan_amount_per_income,months_to_pay_per_month_of_credit_history,PCA_5_0,PCA_5_1,PCA_5_2,PCA_5_3,PCA_5_4
0,Own Home,482087.0,unknown,0.0,11.0,26.3,685960.0,1.0,1.0,debt consolidation,...,0.038023,0.098293,5.076310,24.628703,4.753724,-0.168816,1.080475,-2.203982,1.837586,-1.157517
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,0.0,debt consolidation,...,0.000000,0.385156,4.651241,0.258383,0.078549,0.363500,-0.013454,0.624591,-0.758497,0.175988
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,0.0,debt consolidation,...,0.000000,0.410413,4.587039,15.801155,2.070872,0.992833,-1.463747,0.086640,1.734586,-1.469592
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,1.0,debt consolidation,...,0.044444,0.119064,5.917181,0.150790,0.039656,0.678198,1.350833,-3.014448,0.179879,-1.588368
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,0.0,debt consolidation,...,0.000000,0.120203,9.009564,0.162110,0.107393,-0.075578,1.993978,-1.333405,-0.840971,-0.322678


In [5]:
x = data.drop(columns=['credit_default'])
y = data['credit_default']

x.shape, y.shape

((7500, 30), (7500,))

Делим на тренировочную и валидационную выборку:

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
x_train.shape, x_test.shape

((6000, 30), (1500, 30))